In [1]:
#########################################
# Run once - setup functions            #
# Download .zip files to data folder    #
#########################################
import os
import time

from datetime import datetime, timedelta
import numpy as np
import simplejson, urllib
import pandas as pd
import parameter
import query
import pickle


In [12]:
station_information = simplejson.load(urllib.request.urlopen('https://gbfs.citibikenyc.com/gbfs/en/station_information.json'))

18

In [9]:
station_information = simplejson.load(urllib.request.urlopen('https://gbfs.citibikenyc.com/gbfs/en/station_information.json'))
start_date = datetime(2024, 5, 1, 0, 0, 0)
end_date = datetime(2024, 6, 1, 0, 0, 0)
days = (end_date - start_date).days
weekdays = parameter.get_weekdays_and_weekends(start_date, end_date)[0]
window = timedelta(hours=0.25)
data = {}
valid_stations = []

for file in os.listdir('data/2024/by_station/')
  valid_stations.append(file[:-4])
for file in os.listdir('data/2024/by_station/'):
  now = time.time()
  station = file[:-4]
  df = pd.read_csv('data/2024/by_station/' + file, index_col=0)
  df.dropna(subset=['end_station_name'], inplace=True)
  df['started_at'] = pd.to_datetime(df['started_at'])
  df['ended_at'] = pd.to_datetime(df['ended_at'])
  lat = df['start_lat'].values[0]
  lon = df['start_lng'].values[0]
  data[station] = {}
  data[station]['max_docks'] = station_information['data']['stations'][station]['capacity']
  data[station]['total'] = len(df)
  data[station]['lat'] = lat
  data[station]['lon'] = lon
  data[station]['rate'] = {}
  data[station]['transition'] = {}
  data[station]['dist'] = {}
  for end_station in df['end_station_name'].unique():
    end_df = df[df['end_station_name'] == end_station]
    data[station]['dist'][end_station] = (end_df['ended_at'] - end_df['started_at']).mean()
  print(station, 'total:', data[station]['total'])
  for curr_time in weekdays:
    next_day = curr_time + timedelta(days=1)
    while curr_time < next_day:
      for tick in range(24*4):
        cut_df = df[(df['started_at'] >= curr_time) & (df['started_at'] < curr_time + window)]
        if tick not in data[station]['rate']:
          data[station]['rate'][tick] = 0
          data[station]['transition'][tick] = {}
        data[station]['rate'][tick] += len(cut_df)
        for end_station in cut_df['end_station_name'].unique():
          if end_station not in valid_stations:
            continue
          if end_station not in data[station]['transition'][tick]:
            data[station]['transition'][tick][end_station] = 0
          data[station]['transition'][tick][end_station] += len(cut_df[cut_df['end_station_name'] == end_station])
        curr_time += window
  print('Time taken:', round(time.time() - now, 2))


90 St & 32 Ave total: 370
Time taken: 1.66
31 St & Astoria Blvd total: 1498
Time taken: 1.86
Hicks St & Montague St total: 3165
Time taken: 2.57
Maiden Ln & Pearl St total: 2895
Time taken: 2.88
Concord St & Bridge St total: 1990
Time taken: 2.65
Walton Ave & E 168 St total: 956
Time taken: 1.77
Picnic Point total: 621
Time taken: 1.5
Brooklyn Bridge Park - Pier 2 total: 3212
Time taken: 2.7
Carmine St & 6 Ave total: 8920
Time taken: 5.33
Atlantic Ave & Fort Greene Pl total: 3900
Time taken: 2.66
Plaza St East & Flatbush Ave total: 4542
Time taken: 2.66
E 32 St & Park Ave total: 7040
Time taken: 3.87
Leonard St & Nassau Ave total: 3048
Time taken: 2.39
Irving Ave & DeKalb Ave total: 1275
Time taken: 1.82
44 St & 48 Ave total: 974
Time taken: 1.66
W 51 St & Rockefeller Plaza total: 1334
Time taken: 2.14
W 17 St & 8 Ave total: 4748
Time taken: 2.94
E 43 St & 2 Ave total: 3595
Time taken: 2.72
Ogden Ave & Merriam Ave total: 342
Time taken: 1.43
Walton Ave & E 176 St total: 463
Time taken:

/var/folders/jl/hkqqvvqj0k905cph73s5nsx40000gn/T/ipykernel_55922/3937129953.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ended_at'] = pd.to_datetime(df['ended_at'])


Vesey St & Church St total: 4319
Time taken: 2.56
Jerome Ave & E Mosholu Pkwy S total: 840
Time taken: 1.45
37 Ave & 113 St total: 94
Time taken: 1.22
Jerome Ave & W Fordham Rd total: 925
Time taken: 1.62
Corona Ave & 111 St total: 166
Time taken: 1.28
92 St & 35 Ave total: 210
Time taken: 1.34
Sterling Pl & Classon Ave total: 1402
Time taken: 1.76
Greene St & Prince St total: 3882
Time taken: 2.7
E 106 St & Madison Ave total: 2017
Time taken: 2.05
E 174 St & Walton Ave total: 262
Time taken: 1.47
91 St & 23 Ave total: 128
Time taken: 1.28
21 St & Hoyt Ave S total: 1883
Time taken: 1.93
75 St & Northern Blvd total: 194
Time taken: 1.39
MacDonough St & Marcy Ave total: 1349
Time taken: 1.82
W 160 St & St Nicholas Ave total: 736
Time taken: 1.62
Grand Concourse & E 199 St total: 234
Time taken: 1.37
Willoughby Ave & Onderdonk Ave total: 665
Time taken: 1.45
Page Pl & Grand Ave total: 175
Time taken: 1.36
111 St & Roosevelt Ave total: 244
Time taken: 1.35
3 Ave & E 100 St total: 1538
Time

/var/folders/jl/hkqqvvqj0k905cph73s5nsx40000gn/T/ipykernel_55922/3937129953.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ended_at'] = pd.to_datetime(df['ended_at'])


Ave D & E 3 St total: 3344
Time taken: 2.4
63 St & 59 Dr total: 98
Time taken: 1.19
E 2 St & Ave A total: 5180
Time taken: 3.04
46 St & 30 Rd total: 659
Time taken: 1.52
99 St & 58 Ave total: 163
Time taken: 1.25
W 218 St & Indian Rd total: 334
Time taken: 1.29
E 22 St & 2 Ave total: 5442
Time taken: 2.83
7 Ave & Park Pl total: 5109
Time taken: 2.79
36 St & Hunters Point Ave total: 195
Time taken: 1.27
Reade St & Broadway total: 2916
Time taken: 2.26
Washington Ave & Park Ave total: 3169
Time taken: 2.35
Amsterdam Ave & W 79 St total: 7029
Time taken: 3.18
E 123 St & Lexington Ave total: 1689
Time taken: 1.74
Caton Ave & Bedford Ave total: 1147
Time taken: 1.7
Winthrop St & Schenectady Ave total: 49
Time taken: 1.26
Cadman Plaza E & Tillary St total: 2100
Time taken: 1.93
Brook Ave & E 138 St total: 825
Time taken: 1.44
Madison St & Evergreen Ave total: 767
Time taken: 1.46
53 St & 4 Ave total: 476
Time taken: 1.43
Loring Pl North & W 183 St total: 372
Time taken: 1.37
W 190 St & St Ni

...

Time taken: 1.84
South St & Broad St total: 245
Time taken: 1.42
35 St & 30 Ave total: 1690
Time taken: 1.81
Windsor Pl & Howard Pl total: 1723
Time taken: 1.87
E 161 St & Park Ave total: 777
Time taken: 1.57
President St & Hoyt St total: 1301
Time taken: 2.43
Broadway & W 220 St total: 236
Time taken: 1.38
Coffey St & Conover St total: 864
Time taken: 2.16
5 Ave & E 72 St total: 8172
Time taken: 3.42
W 106 St & Amsterdam Ave total: 3189
Time taken: 2.16
Featherbed Ln & Jerome Ave total: 185
Time taken: 1.32
Frederick Douglass Blvd & Harlem River Dr total: 400
Time taken: 1.37
Ashland Pl & Dekalb Ave total: 1668
Time taken: 2.2
Crescent St & Ditmars Blvd total: 701
Time taken: 1.73
Corona Ave & 102 St total: 98
Time taken: 1.59
27 Ave & 3 St total: 559
Time taken: 1.58
Myrtle Ave & Linden St total: 338
Time taken: 1.87
1 Ave & E 62 St total: 7851
Time taken: 4.79
W 100 St & Manhattan Ave total: 3697
Time taken: 2.79
Hope St & Union Ave total: 4725
Time taken: 3.39
12 St & 4 Ave total: 

/var/folders/jl/hkqqvvqj0k905cph73s5nsx40000gn/T/ipykernel_55922/3937129953.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['started_at'] = pd.to_datetime(df['started_at'])


Time taken: 1.69
St Marks Pl & 4 Ave total: 1220
Time taken: 1.9
Schenectady Ave & Cortelyou Rd total: 96
Time taken: 1.35
Halsey St & Broadway total: 647
Time taken: 1.56
W 12 St & Hudson St total: 5134
Time taken: 2.93
Eastern Pkwy & Brooklyn Ave total: 1258
Time taken: 1.75
Central Park West & W 68 St total: 9605
Time taken: 4.04
E 134 St & Walnut Ave total: 80
Time taken: 1.29
Albany St & Greenwich St total: 3585
Time taken: 2.35
Patchen Ave & Bainbridge St total: 543
Time taken: 1.47
Sterling St & Bedford Ave total: 681
Time taken: 1.49
E 149 St & Park Ave total: 820
Time taken: 1.77
Dwight St & Dikeman St total: 287
Time taken: 1.38
Amsterdam Ave & W 186 St total: 354
Time taken: 1.42
St Marks Ave & Thomas S. Boyland St total: 110
Time taken: 1.3
W 15 St & 10 Ave total: 4386
Time taken: 2.5
Tyler Ave & Maurice Ave total: 272
Time taken: 1.48
56 Ave & 92 St total: 275
Time taken: 1.48
Troy Ave & Linden Blvd total: 183
Time taken: 1.66
Putnam Ave & Knickerbocker Ave total: 758
Time

/var/folders/jl/hkqqvvqj0k905cph73s5nsx40000gn/T/ipykernel_55922/3937129953.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['started_at'] = pd.to_datetime(df['started_at'])


E 12 St & 3 Ave total: 3750
Time taken: 2.51
W 15 St & 6 Ave total: 9088
Time taken: 3.89
7 Ave & Central Park South total: 11733
Time taken: 3.98
E 91 St & Park Ave total: 4425
Time taken: 2.89
7 Ave & 22 St total: 566
Time taken: 1.57
W 43 St & 10 Ave total: 9394
Time taken: 4.41
Jackson Mill Rd & 93 St total: 275
Time taken: 1.46
Brook Ave & E 148 St total: 309
Time taken: 1.56
72 St & 31 Ave total: 301
Time taken: 1.43
8 Ave & W 38 St total: 8661
Time taken: 4.41
Home St & Westchester Ave total: 626
Time taken: 1.62
60 Pl & Bleecker St total: 329
Time taken: 1.5
E 128 St & 3 Ave total: 435
Time taken: 1.54
Vesey Pl & River Terrace total: 9421
Time taken: 4.1


In [10]:
for station_name in data:
  for tick in range(24*4):
    for end_station in data[station_name]['transition'][tick]:
      data[station_name]['transition'][tick][end_station] = np.array(data[station_name]['transition'][tick][end_station]) / sum(data[station_name]['transition'][tick].values())
    data[station_name]['rate'][tick] = np.array(data[station_name]['rate'][tick]) / days
      
with open(f'data/station_data/2024 model/v2/tph_4/data.pickle', 'wb') as handle:
  pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)  

In [16]:
test = {'a': 1, 'b': 2, 'c': 3}
sum(test.values())

6

In [1]:
years = [str(year) for year  in range(2014, 2025)]
months = [str(month) for month in range(1, 13)]
months_name = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

if not os.path.exists("data/"):
  os.makedirs("data/")
  
for year in years:
  path = "data/" + year + "-citibike-tripdata"
  if os.path.exists(path):
    os.rename(path, "data/" + year + "/")
    print(f"Renaming {year}-citibike-tripdata")
  for month in months:
    path = "data/" + year + "/" + month + "_" + months_name[int(month)-1] + "/"
    if os.path.exists(path):
      new_path = "data/" + year + "/" + month + "/"
      os.rename(path, new_path)
      for n in range(1, len(os.listdir(new_path))+1):
        path = new_path + str(int(year)*100+int(month)) + "-citibike-tripdata_" + str(n) + ".csv"
        if os.path.exists(path):
          new_path = new_path + str(n) + ".csv"
          os.rename(path, new_path)          

In [2]:
df = pd.read_csv('data/2024/5/1.csv', low_memory=False)
stations = df['start_station_name'].unique()
pd.DataFrame(stations, columns=['station_name']).to_csv('data/stations.csv')

locations = []
for station in stations:
  if station in df['start_station_name']:
    lat = df.loc[df['start_station_name'] == station]['start_lat'].iloc[0]
    lng = df.loc[df['start_station_name'] == station]['start_lng'].iloc[0]
    locations.append([station, lat, lng])
    print(station, lat, lng)
new_df = pd.DataFrame(locations, columns=['station_name', 'lat', 'lng'])
new_df.to_csv('data/station_locations.csv')

In [2]:
API_KEY = '' # PRIVATE

station_locations = pd.read_csv('data/station_information.csv')
stations = pd.read_csv('data/stations.csv', index_col=0)
stations = stations['station_name'][:10]
done = {station: {} for station in stations}

for orig_station in stations[:5]:
  station_info = station_locations.loc[station_locations['station_name'] == orig_station]
  orig_lat = station_info['lat'].iloc[0]
  orig_lng = station_info['lng'].iloc[0]
  for dest_station in stations:
    if orig_station in done[dest_station]:
      done[orig_station][dest_station] = done[dest_station][orig_station]
    else:
      station_info = station_locations.loc[station_locations['station_name'] == dest_station]
      dest_lat = station_info['lat'].iloc[0]
      dest_lng = station_info['lng'].iloc[0]
      url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={orig_lat},{orig_lng}&destinations={dest_lat},{dest_lng}&mode=biking&language=en-EN&key={API_KEY}"
      result = simplejson.load(urllib.request.urlopen(url))
      if 'duration' in result['rows'][0]['elements'][0]:
        trip_time = result['rows'][0]['elements'][0]['duration']['value']
      else:
        trip_time = np.nan
      done[orig_station][dest_station] = trip_time
          
df = pd.DataFrame(done)
df.to_csv('data/test_dist.csv')
    
    

In [4]:
years = [str(year) for year  in range(2014, 2025)]
months = [str(month) for month in range(1, 13)]
year = '2024'
stations = pd.read_csv('data/stations.csv')
# columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at',
#        'start_station_name', 'start_station_id', 'end_station_name',
#        'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
#        'member_casual']
start = 0 # stopped somewhere around here 
end = len(stations)
window_size = 400

while start < end:
  total_time = time.time()
  end_window = start + window_size
  if start + window_size > end:
    end_window = end
  stations_list = []
  for month in months:
    print('Loading month', month)
    path = f'data/{year}/{month}/'
    if os.path.exists(path):
      for file in os.listdir(path):
        if file.endswith(".csv"):
          now = time.time()
          df = pd.read_csv(path + file, low_memory=False)
          print('Loading dataframe took: ', round(time.time() - now, 4))
          unique_stations = df['start_station_name'].unique()
          stations_to_add = stations['station_name'][start:end_window]
          stations_list.append(df.loc[df['start_station_name'].isin(stations_to_add)])
  now = time.time()
  stations_df = pd.concat(stations_list, ignore_index=True, sort=False)     
  print('Concat took,', round(time.time() - now, 4))
  print("Length of stations_df: ", len(stations_df['start_station_name']))
  if not os.path.exists(f'data/{year}/by_station'):
    os.makedirs(f'data/{year}/by_station')
  print('Saving stations,', stations_df['start_station_name'].unique())
  for station in stations_df['start_station_name'].unique():
    to_save_path = stations_df.loc[stations_df['start_station_name'] == station].reset_index(drop=True)
    to_save_path.to_csv(f'data/{year}/by_station/{station}.csv')
  start = end_window
  print('Loop took,', round(time.time() - total_time, 4))

Loading month 1
Loading month 2
Loading month 3
Loading month 4
Loading month 5
Loading dataframe took:  1.0774
Loading dataframe took:  3.9722
Loading dataframe took:  4.0638
Loading dataframe took:  4.0452
Loading dataframe took:  4.2507
Loading month 6
Loading month 7
Loading month 8
Loading month 9
Loading month 10
Loading month 11
Loading month 12
Concat took, 0.1011
Length of stations_df:  1170604
Saving stations, ['1 Ave & E 110 St' 'E 11 St & Ave B' 'Washington Ave & Empire Blvd'
 'Madison St & Seneca Ave' 'West End Ave & W 107 St' 'W 24 St & 7 Ave'
 'Audubon Ave & W 179 St' 'Allen St & Rivington St'
 'Franklin St & Dupont St' 'Monroe St & Classon Ave'
 'Fulton St & Broadway' 'Van Brunt St & Wolcott St' 'St Marks Pl & 4 Ave'
 'Mercer St & Bleecker St' 'Frost St & Meeker Ave'
 'Grand St & Havemeyer St' 'Washington St & Barrow St' 'Union St & 4 Ave'
 'Lexington Ave & E 24 St' 'E 13 St & 2 Ave' '11 Ave & W 41 St'
 '31 St & Northern Blvd' 'West End Ave & W 60 St'
 'Bedford Ave & Na

In [7]:

all_stations = {}
path = 'data/station_data/2024 model/v1/tph_4/'
save_path = 'data/station_data/2024 model/v1/all_stations_tph_4.pickle'
for file in os.listdir(path):
  if file.endswith(".pickle"):
    all_stations[file[:-7]] = parameter.get_pickle_station(station=file[:-7], path=path)
    
with open(save_path, 'wb') as handle:
    pickle.dump(all_stations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 6, 30, 0, 0, 0)
stations = pd.read_csv('data/stations.csv', index_col=0)
total_station = len(stations)
done_stations = 0
for station_name in stations['station_name']:
    now = time.time()
    station = parameter.get_station(station=station_name, start_date=start_date, end_date=end_date, tph=1, weekday=True)
    parameter.pickle_station(station)
    done_stations += 1
    print(station_name, 'pickled in', round(time.time() - now, 2), 'seconds')
    if done_stations % 10 == 0:
        print(done_stations, 'completed out of', total_station)

yes


In [19]:
station_information = simplejson.load(urllib.request.urlopen('https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json'))
station_status = simplejson.load(urllib.request.urlopen('https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_status.json'))

for station in station_information['data']['stations']:
    print(station['name'], station['lat'], station['lon'])

['Allen St & Stanton St',
 'Carlton Ave & Dean St',
 'W 84 St & Amsterdam Ave',
 'E 85 St & York Ave',
 'Bergen St & 4 Ave',
 'Central Park W & W 91 St',
 'E 15 St & 3 Ave',
 'Riverside Dr & W 78 St',
 'E 44 St & Lexington Ave',
 'Wythe Ave & N 13 St',
 'E 53 St & 3 Ave',
 'Spring St & Hudson St',
 'Willoughby Ave & Tompkins Ave',
 'Bridge St & Water St',
 'W 41 St & 8 Ave',
 'Cooper Square & Astor Pl',
 'Broadway & W 53 St',
 '10 St & 7 Ave',
 'E 7 St & Ave B',
 'Vernon Blvd & 47 Rd',
 '46 Ave & 5 St',
 'Bayard St & Baxter St',
 'Old Slip & South St',
 'Lexington Ave & E 111 St',
 'Macon St & Nostrand Ave',
 'W 54 St & 11 Ave',
 'Clarkson Ave & E 37 St',
 'Grand Concourse & E Mount Eden Ave',
 'Sterling Pl & Franklin Ave',
 'Broadway & W 41 St',
 'Delancey St & Eldridge St',
 'W 111 St & 5 Ave',
 'Clermont Ave & Lafayette Ave',
 '23 Rd & 31 St',
 'Cherry St',
 'Caton Ave & Bedford Ave',
 'W 59 St & 10 Ave',
 'Court St & State St',
 'W Broadway & Watts St',
 'Meserole Ave & Manhattan A

In [15]:
"""
Length of stations_df:  197764
Saving stations, ['Brevoort Pl & Bedford Ave' '19 St & 24 Ave' 'Valentine Ave & E 181 St'
 '5 Ave & E 87 St' 'Henry St & Degraw St' 'Steinway St & 21 Ave'
 'W 163 St & Edgecombe Ave' 'Steinway St & Ditmars Blvd'
 'Lexington Ave & E 120 St' 'Cathedral Pkwy & Broadway'
 'Jerome Ave & Ogden Ave' 'Nelson Ave & 167 St'
 'Louis Nine Blvd & Intervale Ave' 'Aqueduct Ave & W 190 St'
 'Bedford Ave & Fenimore St' '61 St & Borden Ave' '85 St & Northern Blvd'
 'Arthur Ave & E Tremont Ave' 'Washington Ave & Park Pl'
 '60 Ave & Junction Blvd']
"""